# My Implementation

In [32]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, RandomFlip, RandomRotation, RandomZoom, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.activations import relu
import matplotlib.pyplot as plt
from keras.metrics import binary_accuracy
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

seed = 415
batch_size = 8
image_path = "./images"
datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.2,
                              zoom_range = 0.1, # Randomly zoom image
                              width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                              height_shift_range=0.1,
                             rotation_range=30
                             )


# I changed the imagery to grayscale to speed up the training process
train_generator = datagen.flow_from_directory(
    image_path,
    target_size=(227, 227),  # resize for alexnet
    batch_size=batch_size,
    subset='training',
    color_mode="grayscale",
    )

test_generator = datagen.flow_from_directory(
    image_path,
    target_size=(227, 227),  # resize for alexnet
    batch_size=batch_size,
    subset='validation',
    color_mode="grayscale",
    )

train_class_counts = train_generator.classes
test_class_counts = test_generator.classes

train_class_count = dict(zip(train_generator.class_indices.keys(), np.zeros(len(train_generator.class_indices), dtype=int)))
test_class_count = dict(zip(test_generator.class_indices.keys(), np.zeros(len(test_generator.class_indices), dtype=int)))

for label in train_class_counts:
    train_class_count[list(train_generator.class_indices.keys())[int(label)]] += 1

for label in test_class_counts:
    test_class_count[list(test_generator.class_indices.keys())[int(label)]] += 1

print('Number of training samples in each class in the training set:', train_class_count)
print('Number of test samples in each class in the testing set:', test_class_count)

from collections import Counter
counter = Counter(train_generator.classes)
max_val = float(max(counter.values()))
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}
print(class_weights)


Found 17013 images belonging to 3 classes.
Found 4251 images belonging to 3 classes.
Number of training samples in each class in the training set: {'happy': 7192, 'neutral': 4959, 'sad': 4862}
Number of test samples in each class in the testing set: {'happy': 1797, 'neutral': 1239, 'sad': 1215}
{0: 1.0, 1: 1.4502923976608186, 2: 1.479226655697244}


In [33]:
# we'll use this block to build a model that we can tune in keras-tune
import keras_tuner

def build_model(hp:keras_tuner.HyperParameters):

    filter_possibilities = [16, 32, 64, 96, 128, 256, 384, 512]
    dense_size_possibilities = [4, 8, 16, 32, 64, 128, 256, 512, 1024, 4096]
    drop_out_possibilities = list(np.linspace(0.2, 0.6, 10))

    leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.01)

    # hyper parameters we care about
    activation = hp.Choice("activation function", ["relu", "leaky_relu"])
    if activation == "leaky_relu":
        activation = leaky_relu

    first_filter_count = hp.Choice("first filter count", filter_possibilities)
    second_filter_count = hp.Choice("second filter count", filter_possibilities)
    third_filter_count = hp.Choice("third filter count", filter_possibilities)
    fourth_filter_count = hp.Choice("fourth filter count", filter_possibilities)
    fifth_filter_count = hp.Choice("fifth filter count", filter_possibilities)

    first_dense_layer_size = hp.Choice("first dense layer size", dense_size_possibilities)
    second_dense_layer_size = hp.Choice("second dense layer size", dense_size_possibilities)

    first_dense_layer_dropout = hp.Choice("drop out for first dense layer", drop_out_possibilities)
    second_dense_layer_dropout = hp.Choice("dropout for second dense layer", drop_out_possibilities)

    # optimizer
    opt = hp.Choice("optimizer", ["sgd", "adam"])



    model = Sequential([
    # entry point
    Conv2D(filters=first_filter_count,
           kernel_size=(11,11),
           strides=(4,4),
           activation=activation,
           input_shape=(227,227,1)),  # resized for one chanel
    BatchNormalization(),
    MaxPooling2D(pool_size=(3,3), strides=(2,2)),  # extract first feature set


    Conv2D(filters=second_filter_count,
           kernel_size=(5,5), strides=(1,1), activation=activation, padding="same"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3,3), strides=(2,2)),  # extract second feature set


    Conv2D(filters=third_filter_count, kernel_size=(3,3), strides=(1,1), activation=activation, padding="same"),
    BatchNormalization(),
    Conv2D(filters=fourth_filter_count, kernel_size=(3,3), strides=(1,1), activation=activation, padding="same"),
    BatchNormalization(),
    Conv2D(filters=fifth_filter_count, kernel_size=(3,3), strides=(1,1), activation=activation, padding="same"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3,3), strides=(2,2)),  # extract 3rd feature set


    Flatten(),

    # note how this is much smaller than AlexNet
    Dense(first_dense_layer_size, activation=activation),
    Dropout(first_dense_layer_dropout),
    Dense(second_dense_layer_size, activation=activation),
    Dropout(second_dense_layer_dropout),
    Dense(3, activation='softmax'),  # only 3 choices here
    ])


    model.compile(optimizer=opt,
              loss="categorical_crossentropy",
              metrics=['categorical_accuracy']
              )

    model.build(input_shape=(None, 227, 227, 1))
    return model

In [34]:

my_callbacks = [
    EarlyStopping(monitor="val_categorical_accuracy", 
                  patience=50,
                  restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_categorical_accuracy",
                      factor=0.50, patience=50,
                      verbose=1,
                      min_delta=0.0001),
    #ModelCheckpoint(filepath='/content/drive/MyDrive/checkpoints/alex_net_emotions_weighted.{epoch:02d}-{val_loss:.2f}.h5'),
]
build_model(keras_tuner.HyperParameters())

tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective='val_categorical_accuracy',
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory="./models/",
    project_name="emo_detect"
)

print(tuner.search_space_summary())
tuner.search(train_generator,
                    epochs=100,
                    validation_data=test_generator,
                    callbacks=my_callbacks,
                    class_weight=class_weights,
                    verbose=1)

Search space summary
Default search space size: 11
activation function (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'leaky_relu'], 'ordered': False}
first filter count (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64, 96, 128, 256, 384, 512], 'ordered': True}
second filter count (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64, 96, 128, 256, 384, 512], 'ordered': True}
third filter count (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64, 96, 128, 256, 384, 512], 'ordered': True}
fourth filter count (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64, 96, 128, 256, 384, 512], 'ordered': True}
fifth filter count (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64, 96, 128, 256, 384, 512], 'ordered': True}
first dense layer size (Choice)
{'default': 4, 'conditions': [], 'values': [4, 8, 16, 32, 64, 128, 256, 512, 1024, 4096], 'ordered': True}
second dense layer size (Choice)
{'default': 4, 'condi

In [35]:
models = tuner.get_best_models(num_models=5)
print(tuner.results_summary())


Now let's display everything and save the images!

In [36]:
# Evaluate the model on the test set
from contextlib import redirect_stdout

def save_summary(file_path, model):
    with open(file_path, 'w') as f:
        with redirect_stdout(f):
            print(model.summary())

for i, model in enumerate(models):
    model.save(f'./models/model_{i}.h5')
    save_summary(f'./details/summaries/model_{i}.txt', model)

    test_loss, test_acc = model.evaluate(test_generator)
    print('Test accuracy:', test_acc)

    # Make predictions on the test set
    y_pred = model.predict(test_generator)
    y_actual = test_generator.classes
    y_pred = np.argmax(y_pred,axis=1)

    y_pred = np.round(y_pred)

    confusion_mtx = confusion_matrix(y_actual, y_pred)
    print(confusion_mtx)

    # Evaluation
    print(classification_report(test_generator.classes, y_pred))

    plt.imshow(confusion_mtx, cmap='binary', interpolation='nearest')
    plt.colorbar()

    tick_marks = np.arange(3)
    plt.xticks(tick_marks, ['Happy', 'Neutral', 'Sad'], rotation=45)
    plt.yticks(tick_marks, ['Happy', 'Neutral', 'Sad'])

    thresh = confusion_mtx.max() / 2.
    for i in range(confusion_mtx.shape[0]):
        for j in range(confusion_mtx.shape[1]):
            plt.text(j, i, format(confusion_mtx[i, j]), ha="center", va="center", color="white" if confusion_mtx[i, j] > thresh else "black")

    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Model {i} Confusion Matrix')

    plt.show()

    plt.savefig(f'./details/cm_auto_detect/model_{i}_.png')